In [ ]:


# Load your dataset
# Assuming 'df' is the DataFrame that contains your data
# df = pd.read_csv('your_data.csv')




# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Assuming 'df' is the DataFrame that contains your data
# df = pd.read_csv('your_data.csv')


from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

df = df[df['PRODUCT_ID'] == 'Product_001']
# Convert 'ORDER_DATE' to datetime
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])

# Sort data by 'ORDER_DATE' to maintain time series order
df = df.sort_values(by='ORDER_DATE')

# Function to create multiple lag features for ORDER_QTY
def create_lags(df, target_column, num_lags):
    for lag in range(1, num_lags + 1):
        df[f'Lag_{lag}'] = df[target_column].shift(lag)
    return df

# Create lag features for ORDER_QTY, for example, using the last 6 months
num_lags = 6
df = create_lags(df, 'ORDER_QTY', num_lags)

# Drop rows with NaN values (due to shifting from lag features)
df = df.dropna()

# Define the features (Lag_1, Lag_2, ..., Lag_n, UNIT_PRICE, LEAD_TIME_IN_WEEKS) and target (ORDER_QTY)
lag_columns = [f'Lag_{lag}' for lag in range(1, num_lags + 1)]
feature_columns = lag_columns + ['UNIT_PRICE', 'LEAD_TIME_IN_WEEKS']

# Define X (features) and y (target)
X = df[feature_columns]
y = df['ORDER_QTY']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error: {rmse}")

# Calculate R-squared (R²)
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")

# Plot actual vs predicted for the test set
plt.plot(df['ORDER_DATE'].iloc[len(X_train):], y_test, label='Actual', color='blue')
plt.plot(df['ORDER_DATE'].iloc[len(X_train):], y_pred, label='Predicted', color='red')
plt.xlabel('Date')
plt.ylabel('ORDER_QTY')
plt.title('Actual vs Predicted ORDER_QTY')
plt.legend()
plt.show()

# Forecast for the next 25 months
# Create a new DataFrame for future dates
future_dates = pd.date_range(df['ORDER_DATE'].max(), periods=25, freq='M')

# Initialize the last known lag values, UNIT_PRICE, and LEAD_TIME_IN_WEEKS
last_lags = list(df[lag_columns].iloc[-1])
last_unit_price = df['UNIT_PRICE'].iloc[-1]
last_lead_time = df['LEAD_TIME_IN_WEEKS'].iloc[-1]

# Create an empty list to store predictions
future_preds = []

for i in range(25):
    # Prepare input features for the next prediction, including lag values, UNIT_PRICE, and LEAD_TIME_IN_WEEKS
    future_X = np.array([last_lags + [last_unit_price, last_lead_time]])
    
    # Predict the next ORDER_QTY
    future_pred = model.predict(future_X)[0]
    
    # Append the prediction to the list
    future_preds.append(future_pred)
    
    # Update lag values for the next iteration
    last_lags = [future_pred] + last_lags[:-1]  # Shift the lags with the new prediction

# Plot future predictions
plt.plot(future_dates, future_preds, label='Forecasted ORDER_QTY', color='green')
plt.xlabel('Date')
plt.ylabel('ORDER_QTY')
plt.title('ORDER_QTY Forecast for Next 25 Months')
plt.legend()
plt.show()

Mean Squared Error: 20778.750987277377
Root Mean Squared Error: 144.14836449740724
R-squared (R²): 0.7531194024066863
